# Mapa de conexiones y flujo

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
df = pd.read_csv('../data/viajes_surfistas.csv')
df

,homename,homecontinent,homecountry,homelat,homelon,travelcontinent,travelcountry,travellat,travellon
0,Canadian in La La Land,North America,United States of America,33.930030,-118.280993,North America,United States of America,33.887255,-118.411912
1,Canadian in La La Land,North America,United States of America,33.930030,-118.280993,North America,United States of America,34.022100,-118.481000
2,Canadian in La La Land,North America,United States of America,33.930030,-118.280993,North America,United States of America,34.030400,-118.779000
3,Canadian in La La Land,North America,United States of America,33.930030,-118.280993,North America,United States of America,33.887255,-118.411912
4,Canadian in La La Land,North America,United States of America,33.930030,-118.280993,North America,United States of America,33.888889,-118.405278
...,...,...,...,...,...,...,...,...,...
9506,"Bridgetown, Western Australia",Australia,Australia,-33.954448,116.131214,Australia,Australia,-34.225000,115.017000
9507,"Lille, France",Europe,France,50.629250,3.057256,Europe,France,45.002343,-1.201475
9508,MX,South America,Mexico,23.634501,-102.552784,South America,Mexico,21.016700,-101.250000
9509,Kuta - Bali - Indonesia,Asia,Indonesia,-8.739184,115.171130,Asia,Indonesia,-8.710093,115.166845


In [3]:
nodos1 = df[['homecountry', 'homelat', 'homelon']]
nodos1.columns = ['label', 'lat', 'lon'] # renombramos las columnas
# agrupamos por lat y log, no hacerlo por label
nodos1_agg = nodos1.groupby(['lat','lon']).first()
nodos1_agg = nodos1_agg.reset_index()
nodos1_agg

,lat,lon,label
0,-53.773631,-67.707288,Argentina
1,-51.733333,-72.516667,Chile
2,-46.548628,-71.631977,Argentina
3,-45.878760,170.502798,New Zealand
4,-44.885637,168.666159,New Zealand
...,...,...,...
1641,61.924110,25.748151,Finland
1642,62.411363,-149.072971,United States of America
1643,63.264262,10.505392,Norway
1644,64.200841,-149.493673,United States of America


In [4]:
nodos2 = df[['travelcountry', 'travellat', 'travellon']]
nodos2.columns = ['label', 'lat', 'lon'] # renombramos las columnas
# agrupamos por lat y log, no hacerlo por label
nodos2_agg = nodos2.groupby(['lat','lon']).first()
nodos2_agg = nodos2_agg.reset_index()
nodos2_agg

,lat,lon,label
0,-51.055000,-72.832300,Chile
1,-46.592167,-70.932012,Argentina
2,-45.911490,170.490247,New Zealand
3,-45.911290,170.490020,New Zealand
4,-45.904483,170.510135,New Zealand
...,...,...,...
4076,64.133300,-21.933300,Iceland
4077,68.077244,13.474686,Norway
4078,68.207469,14.480151,Norway
4079,68.659089,18.883573,Norway


In [7]:
# unimos los df
nodos = pd.concat([nodos1_agg, nodos2_agg])
nodos = nodos.groupby(['lat','lon']).first()
nodos = nodos.reset_index()
nodos

,lat,lon,label
0,-53.773631,-67.707288,Argentina
1,-51.733333,-72.516667,Chile
2,-51.055000,-72.832300,Chile
3,-46.592167,-70.932012,Argentina
4,-46.548628,-71.631977,Argentina
...,...,...,...
5720,68.077244,13.474686,Norway
5721,68.207469,14.480151,Norway
5722,68.659089,18.883573,Norway
5723,78.217424,15.636744,Norway


In [10]:
# mapa de puntos par ala exploracion de datos
fig = px.scatter_mapbox(
    nodos,
    lat='lat',
    lon='lon',
    mapbox_style='carto-positron',
    zoom=1,
    color='label'
)
fig.update_traces(
    marker={
        'size': 4
    }
)
fig.update_layout( margin={'r':0, 'l':0, 't':0, 'b':0} )
fig.show()

In [11]:
# en plotly hacemos un grafico de lineas pero hacemos salto cada 2 puntos
# lat de donde salgo a la de donde llego y dejar un espacio
# y otra paralela con la lon

lats = np.empty(len(df) * 3)
lats[::3] = df["homelat"]
lats[1::3] = df["travellat"]
lats[2::3] = None



lons = np.empty(len(df) * 3)
lons[::3] = df["homelon"]
lons[1::3] = df["travellon"]
lons[2::3] = None


In [12]:
lats

array([33.9300302, 33.8872549,        nan, ..., -8.7391836, -8.710093 ,
              nan])

In [16]:
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(
    go.Scattermapbox(
        mode='markers+lines',
        lon=lons,
        lat=lats,
        marker = {
            'size':5
        },
        line={
            'width':1
        },
        opacity=0.05
    )

)


fig.update_layout(
    margin={'r':0, 'l':0, 't':0, 'b':0},
    showlegend=False,
    mapbox={
        'style':'carto-darkmatter',
        'zoom':1,
        "center": { 'lon':10, 'lat':10 }
    }
)

fig.show()